In [1]:
import pandas as pd
import numpy as np
import sys
import os
import scipy.io
import re
import turicreate as tc

In [2]:
sub_num = 137
sampling_rate = 1000
ascii_path = '../data/eye_tracking_raw_files/'
trial_order_path = '../data/Trial_order_files/'
trajectory_path = '../data/processed_files/MOT_json_files/'
processed_eye_data = '../data/processed_files/MOT_pre_trial_json_files/MOT_pre_trial_entire_duration_json_files/'
for file in os.listdir('./data/eye_tracking_raw_files/'):
    if str('{0:03}'.format(sub_num)) in file:
        print(file)
        sub_dir = file
ascii_path = ascii_path+sub_dir+'/'

137


In [3]:
ascii_path

'./data/eye_tracking_raw_files/137/'

In [4]:
trial_order_path

'./data/Trial_order_files/'

In [5]:
TARGET_ON = 'TargetOn'
TARGET_OFF = 'TrackingStart'
EFIX = 'EFIX'

def expand_efix(chunk_parts, chunk_list):
    ts_start = int(chunk_parts[2])
    ts_end = int(chunk_parts[3])
    efix_entries = [c for c in chunk_list if int(c[1]) <= ts_end and int(c[1]) >= ts_start]
    for entry in efix_entries:
        entry[len(entry) - 1] = float(0.75)
        #entry[len(entry) - 2] = float(chunk_parts[-1])

In [6]:
rand_p = 0
num_trials_per_block = 30
t = []

for file in os.listdir(ascii_path):
    if '.asc' in file:
        sub_num = file[-9:-6]
        print(sub_num)
        with open(ascii_path+file) as ascii_file:
            start_chunk = False
            chunk_index = 0
            chunk_list = []
            block_num = int(file[-5])
            print(block_num)
            trial_order_mat = scipy.io.loadmat(trial_order_path+'TrialOrder_'+str(sub_num)+'.mat')
            trial_values = trial_order_mat['p'].flatten()[num_trials_per_block*(block_num-1):num_trials_per_block*(block_num)]
            t.append(trial_values)
            print(trial_values)
            for line in ascii_file:
                line = line.replace('\n', '').replace('\r', '').replace('\t', ' ').replace('...', '')
                chunk_parts = [p for p in line.split(' ') if p != '']

                if len(chunk_parts) >= 3:
                    if chunk_parts[2] == TARGET_ON:
                        start_chunk = True
                        rand_p = trial_values[chunk_index]
                        chunk_index += 1
                    elif chunk_parts[2] == TARGET_OFF:
                        start_chunk = False

                if start_chunk == True:
                    if chunk_parts[0] == EFIX:
                        expand_efix(chunk_parts, chunk_list)
                        continue
                    else:
                        try:
                            time_stamp = int(chunk_parts[0])
                            chunk_list.append([rand_p, time_stamp, float(chunk_parts[1]),
                                               float(chunk_parts[2]), float(chunk_parts[3]), float(0.25)])
                        except ValueError:
                            if chunk_parts[0].isdigit() and chunk_parts[1] == '.' and chunk_parts[2] == '.':
                                chunk_list.append([rand_p, int(chunk_parts[0]), -230491, -230491, -230491, float(0.5)])
                            continue

        with open(os.path.splitext(ascii_path+file)[0]+'_pre_tracking_entire_duration.csv', 'w') as csv_file:
            csv_file.write('Trial Number, Timestamp, X, Y, Pupil Area, Fixation flag\n')
            if sampling_rate == 1000:
                for l in chunk_list[::2]:
                    csv_file.write(str(l).replace('[', '').replace(']', '') + '\n')
            else:
                for l in chunk_list:
                    csv_file.write(str(l).replace('[', '').replace(']', '') + '\n')

137
1
[120  96  29  37  19  31  89  88 116 109  32   3 118  52  93  65  13  66
 112   8  98  51 102  27  42  74  57 110  71  75]
137
2
[ 69  59  67 104  48  30   4  83 100  35  97  55  73  78  86  39  25  14
  38 117  44 108  95  77  11  43  53  60 101  90]
137
3
[114   5  64  81  87  94  80  56 103  76   2  45 105  50  26  18  22  49
 106  34  61  28  46  24 107  58  85  47  84  23]
137
4
[ 40  91  16  99  41  12  70  33 113  79  68  20  62 111   1  15  82  72
  21  92  36  10   6  63  17  54 119   7   9 115]


In [7]:
trial_order_mat['p']

array([[120,  96,  29,  37,  19,  31,  89,  88, 116, 109,  32,   3, 118,
         52,  93,  65,  13,  66, 112,   8,  98,  51, 102,  27,  42,  74,
         57, 110,  71,  75,  69,  59,  67, 104,  48,  30,   4,  83, 100,
         35,  97,  55,  73,  78,  86,  39,  25,  14,  38, 117,  44, 108,
         95,  77,  11,  43,  53,  60, 101,  90, 114,   5,  64,  81,  87,
         94,  80,  56, 103,  76,   2,  45, 105,  50,  26,  18,  22,  49,
        106,  34,  61,  28,  46,  24, 107,  58,  85,  47,  84,  23,  40,
         91,  16,  99,  41,  12,  70,  33, 113,  79,  68,  20,  62, 111,
          1,  15,  82,  72,  21,  92,  36,  10,   6,  63,  17,  54, 119,
          7,   9, 115]], dtype=uint8)

In [8]:
num_trials = np.shape(np.unique(trial_order_mat['p']))[0]

In [9]:
sub_dir

'137'

In [10]:
eye_data = tc.SFrame.read_csv(ascii_path+'eye_MOT_'+sub_dir+'_1_pre_tracking_entire_duration.csv', column_type_hints = [int,int,float,float,float, float])
eye_data = eye_data.append(tc.SFrame.read_csv(ascii_path+'eye_MOT_'+sub_dir+'_2_pre_tracking_entire_duration.csv',column_type_hints = [int,int,float,float,float, float]))
eye_data = eye_data.append(tc.SFrame.read_csv(ascii_path+'eye_MOT_'+sub_dir+'_3_pre_tracking_entire_duration.csv',column_type_hints = [int,int,float,float,float, float]))
eye_data = eye_data.append(tc.SFrame.read_csv(ascii_path+'eye_MOT_'+sub_dir+'_4_pre_tracking_entire_duration.csv',column_type_hints = [int,int,float,float,float, float]))
eye_data = eye_data.sort('Trial Number')
#eye_data.explore()

Finished parsing file /Volumes/My Passport/Github/MOT_bayesian_model/data/eye_tracking_raw_files/137/eye_MOT_137_1_pre_tracking_entire_duration.csv

Parsing completed. Parsed 30506 lines in 0.053995 secs.

Finished parsing file /Volumes/My Passport/Github/MOT_bayesian_model/data/eye_tracking_raw_files/137/eye_MOT_137_2_pre_tracking_entire_duration.csv

Parsing completed. Parsed 29484 lines in 0.024596 secs.

Finished parsing file /Volumes/My Passport/Github/MOT_bayesian_model/data/eye_tracking_raw_files/137/eye_MOT_137_3_pre_tracking_entire_duration.csv

Parsing completed. Parsed 30499 lines in 0.025923 secs.

Finished parsing file /Volumes/My Passport/Github/MOT_bayesian_model/data/eye_tracking_raw_files/137/eye_MOT_137_4_pre_tracking_entire_duration.csv

Parsing completed. Parsed 30502 lines in 0.026724 secs.

In [11]:
len(eye_data)/120

1008.2583333333333

In [12]:
len(eye_data['Trial Number'].unique().sort())

119

In [13]:
np.repeat(np.arange(1,num_trials+1),2)

array([  1,   1,   2,   2,   3,   3,   4,   4,   5,   5,   6,   6,   7,
         7,   8,   8,   9,   9,  10,  10,  11,  11,  12,  12,  13,  13,
        14,  14,  15,  15,  16,  16,  17,  17,  18,  18,  19,  19,  20,
        20,  21,  21,  22,  22,  23,  23,  24,  24,  25,  25,  26,  26,
        27,  27,  28,  28,  29,  29,  30,  30,  31,  31,  32,  32,  33,
        33,  34,  34,  35,  35,  36,  36,  37,  37,  38,  38,  39,  39,
        40,  40,  41,  41,  42,  42,  43,  43,  44,  44,  45,  45,  46,
        46,  47,  47,  48,  48,  49,  49,  50,  50,  51,  51,  52,  52,
        53,  53,  54,  54,  55,  55,  56,  56,  57,  57,  58,  58,  59,
        59,  60,  60,  61,  61,  62,  62,  63,  63,  64,  64,  65,  65,
        66,  66,  67,  67,  68,  68,  69,  69,  70,  70,  71,  71,  72,
        72,  73,  73,  74,  74,  75,  75,  76,  76,  77,  77,  78,  78,
        79,  79,  80,  80,  81,  81,  82,  82,  83,  83,  84,  84,  85,
        85,  86,  86,  87,  87,  88,  88,  89,  89,  90,  90,  9

In [14]:
monitor_refresh_rate = 60
eye_tracker_sampling_rate = 500
trial_duration = 2
num_frames_per_trial = int(monitor_refresh_rate * trial_duration)
num_trials = 120 #len(eye_data['Trial Number'].unique())
samples = np.floor(eye_tracker_sampling_rate/monitor_refresh_rate)

In [15]:
downsampled_eye_data = np.zeros([num_frames_per_trial*num_trials,6]) # 6 because of the format of the eye tracking asc file
downsampled_eye_data[:,0] = np.repeat(np.arange(1,num_trials+1),num_frames_per_trial)
for trial in range(1,num_trials+1):
    if trial in eye_data['Trial Number'].unique().sort():
        trial_array = np.zeros([num_frames_per_trial,6])
        trial_df = eye_data[eye_data['Trial Number']==trial][0::samples] # get every nth element from the data frame
        trial_df = trial_df[0:num_frames_per_trial]
        array = trial_df.to_numpy()
        array[array==-230491] = np.nan
        trial_array[0:len(trial_df),:] = array
        trial_array[len(trial_df):,:] = trial_array[len(trial_df)-1,:]
        downsampled_eye_data[(trial-1)*num_frames_per_trial:(trial*num_frames_per_trial),:]=trial_array
    else:
        print(trial)
        downsampled_eye_data[(trial-1)*num_frames_per_trial:(trial*num_frames_per_trial),1:]=np.nan

69


In [16]:
# trial = 32
# eye_data[eye_data['Trial Number']==trial].explore()

In [17]:
def loops_fill(arr):
    out = arr.copy()
    for row_idx in range(out.shape[0]):
        for col_idx in range(1, out.shape[1]): #1st column is trial number, so excluding that column
            if pd.isnull(out[row_idx, col_idx]):
                if out[row_idx,0]==out[row_idx-1,0]:
                    out[row_idx, col_idx] = out[row_idx-1, col_idx]
                else:
                    trial_value = out[row_idx,0]
                    trial_rows = np.where(out==trial_value)[0]
                    col_vals =  np.array(out[np.where(out==trial_value)[0],col_idx],dtype=np.float)
                    print(col_vals)
                    print(np.nanmean(col_vals))
                    out[row_idx, col_idx] = np.nanmean(col_vals)
                    
    return out

In [18]:
downsampled_eye_data

array([[1.0000000e+00, 1.0818693e+07, 6.4640000e+02, 5.6370000e+02,
        1.8000000e+03, 2.5000000e-01],
       [1.0000000e+00, 1.0818683e+07, 6.4630000e+02, 5.6770000e+02,
        1.7990000e+03, 2.5000000e-01],
       [1.0000000e+00, 1.0818667e+07, 6.4470000e+02, 5.6770000e+02,
        1.7880000e+03, 2.5000000e-01],
       ...,
       [1.2000000e+02, 7.9170190e+06, 5.2990000e+02, 3.5820000e+02,
        2.1410000e+03, 7.5000000e-01],
       [1.2000000e+02, 7.9170350e+06, 5.2970000e+02, 3.5830000e+02,
        2.1520000e+03, 7.5000000e-01],
       [1.2000000e+02, 7.9170510e+06, 5.2980000e+02, 3.5490000e+02,
        2.1570000e+03, 7.5000000e-01]])

In [19]:
deye_data = tc.SFrame(downsampled_eye_data).unpack('X1')

deye_data = deye_data.rename(dict(zip(deye_data.column_names(),
                                ['Trial Number','Time stamp','X','Y','Pupil size','Fixation flag'])))
deye_data = deye_data.sort(['Trial Number','Time stamp'])
#downsampled_eye_data.explore()

In [20]:
deye_data_array = deye_data.to_numpy()
processed_array = loops_fill(deye_data_array)
out_eye_data = tc.SFrame(processed_array.astype(float))
out_eye_data = out_eye_data.unpack('X1')
out_eye_data = out_eye_data.rename(dict(zip(out_eye_data.column_names(),
                                ['Trial Number','Time stamp','X','Y','Pupil size','Fixation flag'])))
#eye_data.show()

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
nan
[nan nan nan nan

/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice
  del sys.path[0]
/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: Mean of empty slice
  


In [21]:
trajectory_data = tc.SFrame.read_json(trajectory_path+'behavior test trajectories_0_0_random_with fixation from sub_'+sub_dir)

Parsing JSON records from /Volumes/My Passport/Github/MOT_bayesian_model/data/MOT_json_files/behavior test trajectories_0_0_random_with fixation from sub_137

Successfully parsed an SArray of 72000 elements from the JSON file /Volumes/My Passport/Github/MOT_bayesian_model/data/MOT_json_files/behavior test trajectories_0_0_random_with fixation from sub_137

In [22]:
first_frame_data = trajectory_data[trajectory_data['Frame number']==1]
first_frame_data = first_frame_data.sort(['Trial number'])
first_frame_data_replicate = tc.SFrame()
for i in range(num_frames_per_trial):
    first_frame_data_replicate = first_frame_data_replicate.append(first_frame_data)
print(len(first_frame_data_replicate))
first_frame_data_replicate = first_frame_data_replicate.sort(['Trial number','Frame number'])

14400


In [23]:
first_frame_data_replicate['Time stamp'] = out_eye_data['Time stamp']
first_frame_data_replicate['fixation.x'] = out_eye_data['X']
first_frame_data_replicate['fixation.y'] = out_eye_data['Y']
first_frame_data_replicate['Pupil size'] = out_eye_data['Pupil size'].astype(int)
first_frame_data_replicate['Fixation flag'] = out_eye_data['Fixation flag']

In [24]:
#first_frame_data_replicate[first_frame_data_replicate['Trial number']==1].explore()

In [25]:
df = first_frame_data_replicate.to_dataframe()
print(len(df[df.isnull().T.any().T]))
df[df.isnull().T.any().T]['Trial number'].unique()

120


array([69])

In [26]:
print(len(first_frame_data_replicate))
first_frame_data_replicate = first_frame_data_replicate.dropna('fixation.x',how='all')
print(len(first_frame_data_replicate))

14400
14280


In [27]:
temp = tc.SFrame.read_json(processed_eye_data+'sub_105'+'_pre_tracking_entire_duration_with_trajectory_data',orient='records')
#temp.explore()

Parsing JSON records from /Volumes/My Passport/Github/MOT_bayesian_model/data/MOT_pre_trial_json_files/MOT_pre_trial_entire_duration_json_files/sub_105_pre_tracking_entire_duration_with_trajectory_data

Successfully parsed an SArray of 14160 elements from the JSON file /Volumes/My Passport/Github/MOT_bayesian_model/data/MOT_pre_trial_json_files/MOT_pre_trial_entire_duration_json_files/sub_105_pre_tracking_entire_duration_with_trajectory_data

In [28]:
first_frame_data_replicate.export_json(processed_eye_data+'sub_'+sub_num+'_pre_tracking_entire_duration_with_trajectory_data',orient='records')

In [29]:
processed_eye_data+'sub_'+sub_num+'_pre_tracking'

'./data/MOT_pre_trial_json_files/MOT_pre_trial_entire_duration_json_files/sub_137_pre_tracking'

In [30]:
processed_eye_data

'./data/MOT_pre_trial_json_files/MOT_pre_trial_entire_duration_json_files/'

In [31]:
print(first_frame_data_replicate.column_names())
print(first_frame_data_replicate.column_types())

['Frame number', 'Number of Targets', 'Object 01.x', 'Object 01.y', 'Object 02.x', 'Object 02.y', 'Object 03.x', 'Object 03.y', 'Object 04.x', 'Object 04.y', 'Object 05.x', 'Object 05.y', 'Object 06.x', 'Object 06.y', 'Object 07.x', 'Object 07.y', 'Object 08.x', 'Object 08.y', 'Object 09.x', 'Object 09.y', 'Object 10.x', 'Object 10.y', 'Object 11.x', 'Object 11.y', 'Object 12.x', 'Object 12.y', 'Object 13.x', 'Object 13.y', 'Object 14.x', 'Object 14.y', 'Object 15.x', 'Object 15.y', 'Object 16.x', 'Object 16.y', 'Speed (deg/sec)', 'Subject ID', 'Trial number', 'fixation.x', 'fixation.y', 'Time stamp', 'Pupil size', 'Fixation flag']
[<class 'int'>, <class 'int'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class

In [32]:
len(out_eye_data['Trial Number'].unique())

120

In [33]:
trial_data = first_frame_data_replicate[first_frame_data_replicate['Trial number'] ==25]
for name in trial_data.column_names():
    if not trial_data[name].all(): # Remove all empty columns
        trial_data = trial_data.remove_column(name)
trial_data.explore()
print(trial_data.column_names())

Materializing SFrame

['Frame number', 'Number of Targets', 'Object 01.x', 'Object 01.y', 'Object 02.x', 'Object 02.y', 'Object 03.x', 'Object 03.y', 'Object 04.x', 'Object 04.y', 'Object 05.x', 'Object 05.y', 'Object 06.x', 'Object 06.y', 'Object 07.x', 'Object 07.y', 'Object 08.x', 'Object 08.y', 'Object 09.x', 'Object 09.y', 'Object 10.x', 'Object 10.y', 'Object 11.x', 'Object 11.y', 'Object 12.x', 'Object 12.y', 'Object 13.x', 'Object 13.y', 'Object 14.x', 'Object 14.y', 'Speed (deg/sec)', 'Subject ID', 'Trial number', 'fixation.x', 'fixation.y', 'Time stamp', 'Pupil size', 'Fixation flag']


In [34]:
trial_data['fixation.y'].all()

True

In [36]:
temp = tc.SFrame.read_json('./data/testing_output/correspondences_sub_20_Hz_128.json')
temp.explore()

Parsing JSON records from /Volumes/My Passport/Github/MOT_bayesian_model/data/testing_output/correspondences_sub_20_Hz_128.json

Successfully parsed an SArray of 120 elements from the JSON file /Volumes/My Passport/Github/MOT_bayesian_model/data/testing_output/correspondences_sub_20_Hz_128.json

Materializing SFrame